# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [ ]:
# Install any packages that you might need
!pip install smdebug --upgrade

In [ ]:
# Import all necessary packages
import sagemaker
from sagemaker.session import Session
import boto3
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [ ]:
!rm -r .dogImages/train/.ipynb_checkpoints
!rm -r .dogImages/valid/.ipynb_checkpoints
!rm -r .dogImages/test/.ipynb_checkpoints

In [ ]:
basedir = './dogImages'

arr_train = []
arr_valid = []
arr_test = []

for subdir, dirs, files in os.walk(basedir):
    for file in files:
        path_arr = subdir.split('/')
        folder = path_arr[2]
        label = int(path_arr[3].split('.')[0])
        if (folder == 'train'):
            arr_train.append([label, subdir, file])
        elif (folder == 'valid'):
            arr_valid.append([label, subdir, file])
        elif (folder == 'test'):
            arr_test.append([label, subdir, file])

df_train = pd.DataFrame(arr_train, columns = ['label', 'subdir', 'file']).sort_values(['label', 'file'], ignore_index = True)
df_train['row'] = range(len(df_train))

df_valid = pd.DataFrame(arr_valid, columns = ['label', 'subdir', 'file']).sort_values(['label', 'file'], ignore_index = True)
df_valid['row'] = range(len(df_valid))

df_test = pd.DataFrame(arr_test, columns = ['label', 'subdir', 'file']).sort_values(['label', 'file'], ignore_index = True)
df_test['row'] = range(len(df_test))

In [ ]:
# Display number of classes
num_classes = df_train['label'].nunique()
print(f'There are {num_classes} classes in this dataset.')

# Examine a sample of rows of each dataframe
print(f'There are {len(df_train.index)} training images.')
df_train.sample(n = 10, random_state = 1)
print(f'There are {len(df_valid.index)} validation images.')
df_valid.sample(n = 10, random_state = 1)
print(f'There are {len(df_test.index)} testing images.')
df_test.sample(n = 10, random_state = 1)

In [ ]:
df_breed_labels = df_train[['label', 'subdir']].copy().drop_duplicates().rename(columns = {'subdir': 'breed'}).set_index('label')
df_breed_labels['breed'] = df_breed_labels['breed'].apply(lambda row: row.split('/')[-1].split('.')[-1].replace('_', ' '))
df_breed_labels.head()

In [ ]:
# Set up variables related to AWS account
session = sagemaker.Session()

bucket = session.default_bucket()
print(f'Default Bucket: {bucket}')

region = session.boto_region_name
print(f'AWS Region: {region}')

role = sagemaker.get_execution_role()
print(f'RoleArn: {role}')

In [ ]:
# Upload dog images to S3
os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages/train/ s3://${DEFAULT_S3_BUCKET}/data/train/
!aws s3 sync ./dogImages/valid/ s3://${DEFAULT_S3_BUCKET}/data/valid/
!aws s3 sync ./dogImages/test/ s3://${DEFAULT_S3_BUCKET}/data/test/

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#TODO: Declare your HP ranges, metrics etc.

In [ ]:
#TODO: Create estimators for your HPs

estimator = # TODO: Your estimator here

tuner = # TODO: Your HP tuner here

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit() # TODO: Remember to include your data channels

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()